# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [12]:
import pandas as pd
import numpy as np
from scipy import stats


In [14]:
data = pd.io.stata.read_stata('us_job_market_discrimination.dta')

In [15]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [16]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


Question: 1. What test is appropriate for this problem? Does CLT apply?
The statement is about comparing two segments and their proportion of success and failures. It is therefore appropriate to use significance testing as we are comparing the two proportions.

Central limit theorem applies in this case. As long as we have a discrete distribution.

Question: 2. What are the null and alternate hypotheses?

Null Hypothesis: Proportion(white call) = Proportion(black call)
Alternative Hypothesis: Proportion(white call) != Proportion(black call)
We can interpret the null hypothesis as the difference between the two success proportions is zero.

In [17]:
w = data[data.race=='w']
b = data[data.race=='b']

In [7]:
# Question: 3. Compute margin of error, confidence interval, and p-value. 

In [18]:
import scipy.stats as stats

alpha = 0.05

w_size = len(w)
b_size = len(b)

w_successrate = len(w[w["call"] == 1]) / w_size
b_successrate = len(b[b["call"] == 1]) / b_size

print("Critical value: %.2f" % alpha)
print("Proportion of white call: %.2f%%" \
      % (w_successrate*100))
print("Proportion of black call: %.2f%%" \
      % (b_successrate*100))

Critical value: 0.05
Proportion of white call: 9.65%
Proportion of black call: 6.45%


In [20]:
prop = (w_successrate*w_size + b_successrate*b_size) \
                / (w_size + b_size)

SError = (prop * (1-prop) \
     * ((1/w_size) + (1/b_size)))**0.5

z = (w_successrate - b_successrate) / SError

print("Sample proportion: %.3f%%" % prop)
print("Standard Error: %.3f%%" % SError)
print("Z-statistic: %.3f%%" % z)

Sample proportion: 0.080%
Standard Error: 0.008%
Z-statistic: 4.108%


In [21]:
p = stats.norm.sf(abs(z))

print("P-value: %.5f\n" % p)
print("P-Value greater than alpha? %s" % str(p > alpha))
if p > alpha:
    print("Proportion of the segments are NOT different")
else:
    print("Proportion of the segments ARE different")

P-value: 0.00002

P-Value greater than alpha? False
Proportion of the segments ARE different


In [22]:
prop_diff = w_successrate - b_successrate

print("Difference in proportion between the segments : %.1f%%" \
      % (prop_diff*100))

Difference in proportion between the segments : 3.2%


In [25]:
conf_level = 0.95

dist = stats.norm.ppf((1+conf_level)/2)
std_dev_diff = ((w_successrate *(1-w_successrate))/w_size \
                + (b_successrate *(1-b_successrate))/b_size)**0.5
conf_int = dist * std_dev_diff

print("Confidence Interval: %.3f\n" % conf_int)
print("True proportion difference is between %.2f%% and %.2f%%." \
      % ((prop_diff - conf_int)*100, (prop_diff + conf_int)*100))


Confidence Interval: 0.015

True proportion difference is between 1.68% and 4.73%.
We will take the bootstrap approach. We will test our hypothesis by assuming that our the null hypothesis is true.


In [26]:
prop_diff = w_successrate - b_successrate

np.random.seed(42)
S = []
for i in range(10000):
    rand_ind = np.random.randint(0, len(data), len(data))
    perm_data = data["call"].iloc[rand_ind]
    perm_w = perm_data[:len(w)]
    perm_b = perm_data[len(b):]
    perm_w_mean = np.mean(perm_w)
    perm_b_mean = np.mean(perm_b)
    S.append(perm_w_mean - perm_b_mean)
S = np.array(S)

p = np.sum(S >= prop_diff)/10000
print("P-value: %.5f\n" % p)
print("P-Value larger than alpha? %s" % str(p > alpha))
if p > alpha:
    print("Proportion of the segments are NOT different")
else:
    print("Proportion of the segments ARE different")

P-value: 0.00000

P-Value larger than alpha? False
Proportion of the segments ARE different


In [27]:
#95% Confidence interval

conf_int = np.percentile(S, [2.5, 97.5])

print("Confidence Interval: %.3f\n" % conf_int[1])
print("True proportion difference is between %.2f%% and %.2f%%." \
      % ((prop_diff + conf_int[0])*100, (prop_diff + conf_int[1])*100))

Confidence Interval: 0.015

True proportion difference is between 1.64% and 4.72%.


4. Write a story describing the statistical significance in the context or the original problem.
The person with white-sounding names have the probability of 9.65% to get the call. Whereas, with black-sounding names, the probability is only 6.45%. The difference in the mean is 3.2% which is quite significant. In the 10,000 trials we performed assuming our null hypotheises is true, we didnot got any reading that was that high. So our test shows us that their is a discrimination in the process. Our random sampling proofs the racial bias.

5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?
As I test the difference between the other segments of the data it appears to be random. It further inhance our result that their is a discrimination in the hiring process. The person with white sounding name are more likely to get a callback.

In [28]:
honors = np.mean(w["honors"]) - np.mean(b["honors"])
education = np.mean(w["education"]) - np.mean(b["education"])
experience = np.mean(w["yearsexp"]) - np.mean(b["yearsexp"])
volunteer = np.mean(w["volunteer"]) - np.mean(b["volunteer"])
occupspecific = np.mean(w["occupspecific"]) - np.mean(b["occupspecific"])

print("Difference in `honors`: %.2f%%" \
      % (honors*100))
print("Difference in `education`: %.2f%%" \
      % (education*100))
print("Difference in `experience`: %.2f%%" \
      % (experience*100))
print("Difference in `volunteer`: %.2f%%" \
      % (volunteer*100))
print("Difference in `occupspecific`: %.2f%%" \
      % (occupspecific*100))

Difference in `honors`: 0.29%
Difference in `education`: 0.49%
Difference in `experience`: 2.67%
Difference in `volunteer`: -0.57%
Difference in `occupspecific`: -221.44%
